# Discriminative Biclustering Algorithm 
Proposed by Odibat & Reddy, 2014 in **Efficient mining of discriminative co-clusters from gene
expression data**

In [6]:
%load_ext pycodestyle_magic

In [33]:
%matplotlib inline
import numpy as np
import math
from matplotlib import pyplot as plt
from sklearn.metrics import consensus_score

In [12]:
# !pip install pycodestyle
# !pip install pycodestyle_magic

### Definition 1 - Coherence Measure H

In [55]:
#%pycodestyle
class CoherenceMeasure(object):
    def __init__(self, data):
        self.data = data
        self.n, self.m = data.shape
        self.xiJ = np.mean(data, axis=1)
        self.xIj = np.mean(data, axis=0)
        self.xIJ = np.mean(data)
        self._H = None

    def _compute_H(self):
        H = 0
        for i in range(self.n):
            for j in range(self.m):
                H += (self.data[i, j] - self.xIj[j] -
                      self.xiJ[i] + self.xIJ)**2
        H *= 1.0/math.fabs(self.m*self.n)
        return 1-H

Loading test data for Coherence Measure

In [56]:
import random
data = np.random.random((50, 50))
print(data)

[[ 0.80846514  0.7185603   0.1889832  ...,  0.89627401  0.54641834
   0.49237191]
 [ 0.90087431  0.47441586  0.81390441 ...,  0.41829814  0.30073035
   0.41904815]
 [ 0.46041853  0.05210407  0.17666657 ...,  0.9119626   0.97997355
   0.3882357 ]
 ..., 
 [ 0.35750083  0.51813385  0.81431036 ...,  0.69363711  0.74477207
   0.77208623]
 [ 0.57406462  0.01223324  0.05346187 ...,  0.296153    0.98559784
   0.25599924]
 [ 0.66573601  0.27202778  0.16723127 ...,  0.05726635  0.93252375
   0.07560622]]


In [58]:
# Testing Coherence
coherence_measure = CoherenceMeasure(data)
coherence_measure._compute_H()

0.9202906264400289